# Final small business loan approval  

This notebook marks the final loan approval work around for my final year capstone project.

## problem definition:
The case-study project: we assume the role of loan officer at a bank and try to approve or deny a loan by assessing its risk of default using machine learning models. The most accurate model will be picked for deployment and integration with an interface for interactive decision making.

## project objective:
The project will try to answer the following questions: As a representative of the bank, should I grant a loan to a particular small business (Company X)? Why or why not? The decision made by assessing a loan's risk.

generally:
The assessment is accomplished by estimating the loan's default probability through analyzing the historical dataset and then classifying the loan into one of two categories: (a) higher risk—likely to default on the loan (i.e., be charged off/failure to pay in full) or (b) lower risk—likely to pay off the loan in full. There have been many success stories of start-ups receiving SBA loan guarantees such as FedEx and Apple Computer. However, there have also been stories of small businesses and/or start-ups that have defaulted on their SBA-guaranteed loans. 

## Dataset:
The dataset used in this project is the U.S. Small Business Administration (SBA). Here is the link to the dataset- [Link Text](https://www.kaggle.com/datasets/mirbektoktogaraev/should-this-loan-be-approved-or-denied) 

The dataset is a real dataset from the U.S. Small Business Administration (SBA). The case-study assignment, titled “Should This Loan be Approved or Denied?” is designed to teach statistical thinking by focusing on how to use real data to make informed decisions for a particular purpose. For this assignment, students assume the role of a loan officer who is deciding whether to approve a loan to a small business.